In [1]:
from collections import Counter
import random

import numpy as np
import pyarrow.dataset as ds
import duckdb
import pandas as pd
import numpy as np
import seaborn as sns
import plotly as pl
import plotly.express as px
import json


INDICADORES = {
    "AFD": "Adequação da Formação Docente",
    "ICG": "Complexidade de Gestão da Escola",
    "IED": "Esforço Docente",
    "ATU": "Média de Alunos por Turma",
    "HAD": "Média de Horas-aula diária",
    "DSU": "Percentual de Docentes com Curso Superior",
    "TDI": "Taxas de Distorção Idade-série",
            "TRE": "Taxa de Rendimento Escolar",

}

DIMENSIONS = {
    "Ano": "NU_ANO_CENSO",
    "Nome da região geográfica": "NO_REGIAO",
    "Nome da Unidade da Federação": "NO_UF",
    "Nome da Mesorregião": "NO_MESORREGIAO",
    "Nome da Microrregião": "NO_MICRORREGIAO"
}

with open("../map_indicadores.json") as f:
    MAP_INDICADORES = json.load(f)

In [68]:
year = 2016
indicador = "TRE"
file = f"../../../data/raw/{indicador}/{year}.xlsx"
df = pd.read_excel(file, skiprows=8, skipfooter=6, na_values=["--"])

In [60]:
a = df.iloc[:,:9]

In [69]:
b = df.iloc[:,9:27]

In [70]:
c = pd.concat([a,b], axis=1)

In [72]:
c.columns[:9]

Index(['Ano', 'NO_REGIAO', 'SG_UF', 'CO_MUNICIPIO', 'NO_MUNICIPIO',
       'CO_ENTIDADE', 'NO_ENTIDADE', 'TIPOLOCA', 'Dependad'],
      dtype='object')

In [ ]:
columns = ['NU_ANO_CENSO', 'NO_REGIAO', 'SG_UF', 'CO_MUNICIPIO', 'NO_MUNICIPIO',
           'CO_ENTIDADE', 'NO_ENTIDADE', 'NO_CATEGORIA', 'NO_DEPENDENCIA']
df = df.rename(
    columns=dict(zip(df.columns[:9], columns))
)

In [23]:

# renamed columns
columns = ['NU_ANO_CENSO', 'NO_REGIAO', 'SG_UF', 'CO_MUNICIPIO', 'NO_MUNICIPIO',
           'CO_ENTIDADE', 'NO_ENTIDADE', 'NO_CATEGORIA', 'NO_DEPENDENCIA']
df2 = df.rename(
    columns=dict(zip(df.columns[:9], columns))
)

# deleted columns
df2 = df.drop(columns=["NO_ENTIDADE"])

df2.insert(8, "NO_INDICADOR", INDICADORES[indicador])
df2.insert(9, "SG_INDICADOR", indicador)


In [24]:
list(enumerate(df2.columns))

[(0, 'Ano'),
 (1, 'NO_REGIAO'),
 (2, 'SIGLA'),
 (3, 'PK_COD_MUNICIPIO'),
 (4, 'NO_MUNICIPIO'),
 (5, 'PK_COD_ENTIDADE'),
 (6, 'TIPOLOCA'),
 (7, 'Dependad'),
 (8, 'NO_INDICADOR'),
 (9, 'SG_INDICADOR'),
 (10, 'AFD_INF1'),
 (11, 'AFD_INF2'),
 (12, 'AFD_INF3'),
 (13, 'AFD_INF4'),
 (14, 'AFD_INF5'),
 (15, 'AFD_FUN1'),
 (16, 'AFD_FUN2'),
 (17, 'AFD_FUN3'),
 (18, 'AFD_FUN4'),
 (19, 'AFD_FUN5'),
 (20, 'AFD_F141'),
 (21, 'AFD_F142'),
 (22, 'AFD_F143'),
 (23, 'AFD_F144'),
 (24, 'AFD_F145'),
 (25, 'AFD_F581'),
 (26, 'AFD_F582'),
 (27, 'AFD_F583'),
 (28, 'AFD_F584'),
 (29, 'AFD_F585'),
 (30, 'AFD_MED1'),
 (31, 'AFD_MED2'),
 (32, 'AFD_MED3'),
 (33, 'AFD_MED4'),
 (34, 'AFD_MED5'),
 (35, 'AFD_EJAF1'),
 (36, 'AFD_EJAF2'),
 (37, 'AFD_EJAF3'),
 (38, 'AFD_EJAF4'),
 (39, 'AFD_EJAF5'),
 (40, 'AFD_EJAM1'),
 (41, 'AFD_EJAM2'),
 (42, 'AFD_EJAM3'),
 (43, 'AFD_EJAM4'),
 (44, 'AFD_EJAM5')]

In [28]:
id_vars = df2.columns[:10]
value_vars = df2.columns[10:]
df3 = df2.melt(id_vars=id_vars, value_vars=value_vars, var_name="TP_GRUPO", value_name="METRICA")

In [29]:
df3["METRICA"].astype("float")

0           NaN
1           NaN
2           NaN
3          50.0
4           NaN
           ... 
6392115     NaN
6392116     NaN
6392117     NaN
6392118     NaN
6392119     NaN
Name: METRICA, Length: 6392120, dtype: float64